In [54]:
import replicate
from datasets import load_dataset

import json
import jsonlines

In [67]:
%cd pass@k_eval

/Users/sangmin/Desktop/Dev/Korea_Univ/short-coding-LLM/pass@k_eval


In [3]:
from getpass import getpass
import os

REPLICATE_API_TOKEN = getpass()
os.environ['REPLICATE_API_TOKEN'] = REPLICATE_API_TOKEN

In [55]:
model = replicate.models.get("meta/codellama-34b-instruct")
version = model.versions.get("b17fdb44c843000741367ae3d73e2bb710d7428a662238ddebbf4302db2b5422")

In [385]:
params = {
    "top_k": 50,
    "top_p": 1.0,
    "max_tokens": 300,
    "temperature": 0,
    "system_prompt": "Responses should be written in Python.",
    "repeat_penalty": 1.1,
    "presence_penalty": 0,
    "frequency_penalty": 0
}

# Code Summarization: 2-stage
(1) 함수를 구현하고, (2) 구현 결과를 다시 프롬프팅하여 코드를 축약

In [68]:
dataset_pred = load_dataset("json", data_files="predictions/prediction_instruct_baseline.jsonl", split="train")
dataset_pred

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /Users/sangmin/.cache/huggingface/datasets/json/default-a549b1e62565d4d7/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


Dataset({
    features: ['task_id', 'completion'],
    num_rows: 164
})

In [69]:
template = """Rewrite the code below as short as possible.
[PYTHON]
{}
[/PYTHON]"""
print(template.format(dataset_pred[123]["completion"]))

Rewrite the code below as short as possible.
[PYTHON]
def get_odd_collatz(n):
    if n <= 0:
        raise ValueError("n should be a positive integer.")
    
    collatz_list = [n]
    while n != 1:
        if n % 2 == 0:
            n //= 2
        else:
            n = 3 * n + 1
        collatz_list.append(n)
    
    return sorted([x for x in collatz_list if x % 2 != 0])
[/PYTHON]


In [14]:
predictions = []
for example in dataset_pred:
    prediction = replicate.predictions.create(
        version=version,
        input={
                "top_k": 50,
                "top_p": 1.0,
                "max_tokens": 500,
                "temperature": 0,
                "system_prompt": "Responses should be written in Python.",
                "prompt": template.format(example["completion"]),
                "repeat_penalty": 1.1,
                "presence_penalty": 0,
                "frequency_penalty": 0
            },
    )
    predictions.append(prediction)

# Code Summarization: 2-stage (1-shot)

## Try 1:

In [132]:
template = """Rewrite the code as short as possible.
[PYTHON]
{completion}
[/PYTHON]
Just refer to the example of code condensation with the delimiter #.
[PYTHON]
{example}
[/PYTHON]
"""

In [237]:
example = """#Example
##input
def minLengthAfterRemovals(self, nums: List[int]) -> int:
    counter = Counter(nums)
    heap = [ -cnt for cnt in counter.values()]
    heapify(heap)
    while len(heap) > 1:
        largest = -heappop(heap)
        second = -heappop(heap)
        if largest > 1:
            heappush(heap, 1-largest)
        if second > 1:
            heappush(heap, 1-second) 
    return -heap[0] if heap else 0
##output
def minLengthAfterRemovals(self, nums: List[int]) -> int:
		scount = Counter(nums)
		arr = sorted([scount[num] for num in sorted(scount.keys())], key=lambda x: -x)    
		
		return (arr[0] - sum(arr[1:])) if arr[0] > sum(arr[1:]) else sum(arr) % 2
"""
print(example)

#Example
##input
def minLengthAfterRemovals(self, nums: List[int]) -> int:
    counter = Counter(nums)
    heap = [ -cnt for cnt in counter.values()]
    heapify(heap)
    while len(heap) > 1:
        largest = -heappop(heap)
        second = -heappop(heap)
        if largest > 1:
            heappush(heap, 1-largest)
        if second > 1:
            heappush(heap, 1-second) 
    return -heap[0] if heap else 0
##output
def minLengthAfterRemovals(self, nums: List[int]) -> int:
		scount = Counter(nums)
		arr = sorted([scount[num] for num in sorted(scount.keys())], key=lambda x: -x)    
		
		return (arr[0] - sum(arr[1:])) if arr[0] > sum(arr[1:]) else sum(arr) % 2



## Try 2

In [241]:
template = """
Your task is to condense Python function that must be between the [PYTHON] and [\PYTHON] tag. Do not change the function name. Please do not repeat. Here is an example :
{example}
problem :
[PYTHON]
{completion}
[/PYTHON]
"""

In [270]:
example = """def minLengthAfterRemovals(self, nums: List[int]) -> int:
    counter = Counter(nums)
    heap = [ -cnt for cnt in counter.values()]
    heapify(heap)
    while len(heap) > 1:
        largest = -heappop(heap)
        second = -heappop(heap)
        if largest > 1:
            heappush(heap, 1-largest)
        if second > 1:
            heappush(heap, 1-second) 
    return -heap[0] if heap else 0
def minLengthAfterRemovals(self, nums: List[int]) -> int:
    scount = Counter(nums)
    arr = sorted([scount[num] for num in sorted(scount.keys())], key=lambda x: -x)    
    
    return (arr[0] - sum(arr[1:])) if arr[0] > sum(arr[1:]) else sum(arr) % 2"""

print(example)

def minLengthAfterRemovals(self, nums: List[int]) -> int:
    counter = Counter(nums)
    heap = [ -cnt for cnt in counter.values()]
    heapify(heap)
    while len(heap) > 1:
        largest = -heappop(heap)
        second = -heappop(heap)
        if largest > 1:
            heappush(heap, 1-largest)
        if second > 1:
            heappush(heap, 1-second) 
    return -heap[0] if heap else 0
def minLengthAfterRemovals(self, nums: List[int]) -> int:
    scount = Counter(nums)
    arr = sorted([scount[num] for num in sorted(scount.keys())], key=lambda x: -x)    
    
    return (arr[0] - sum(arr[1:])) if arr[0] > sum(arr[1:]) else sum(arr) % 2


## Try 3:

In [282]:
before = """def minLengthAfterRemovals(self, nums: List[int]) -> int:
    counter = Counter(nums)
    heap = [ -cnt for cnt in counter.values()]
    heapify(heap)
    while len(heap) > 1:
        largest = -heappop(heap)
        second = -heappop(heap)
        if largest > 1:
            heappush(heap, 1-largest)
        if second > 1:
            heappush(heap, 1-second) 
    return -heap[0] if heap else 0
"""

after = """def minLengthAfterRemovals(self, nums: List[int]) -> int:
    scount = Counter(nums)
    arr = sorted([scount[num] for num in sorted(scount.keys())], key=lambda x: -x)    
    
    return (arr[0] - sum(arr[1:])) if arr[0] > sum(arr[1:]) else sum(arr) % 2"""

example = {
    "before": before,
    "after": after
}

In [370]:
template = """Your task is to rewrite a Python function in a shorter form. The function name and parameters must be the same as before. 
Outputs must be between the [PYTHON] and [\PYTHON] tags.

You are given one example. 

Input:
{before}
Output:
{after}

Input:
{completion}

Output:
"""

In [371]:
print(template.format(before=example["before"], after=example["after"], completion=dataset_pred[-1]["completion"]))

Your task is to rewrite a Python function in a shorter form. The function name and parameters must be the same as before. 
Outputs must be between the [PYTHON] and [\PYTHON] tags.

You are given one example. 

Input:
def minLengthAfterRemovals(self, nums: List[int]) -> int:
    counter = Counter(nums)
    heap = [ -cnt for cnt in counter.values()]
    heapify(heap)
    while len(heap) > 1:
        largest = -heappop(heap)
        second = -heappop(heap)
        if largest > 1:
            heappush(heap, 1-largest)
        if second > 1:
            heappush(heap, 1-second) 
    return -heap[0] if heap else 0

Output:
def minLengthAfterRemovals(self, nums: List[int]) -> int:
    scount = Counter(nums)
    arr = sorted([scount[num] for num in sorted(scount.keys())], key=lambda x: -x)    
    
    return (arr[0] - sum(arr[1:])) if arr[0] > sum(arr[1:]) else sum(arr) % 2

Input:
def generate_integers(a, b):
    if a > b:
        a, b = b, a
    even_digits = []
    for i in range(a, b+1):


In [372]:
predictions = []
count = 0
for data in dataset_pred:
    prediction = replicate.predictions.create(
        version=version,
        input={
                **params,
                "prompt": template.format(before=example["before"], after=example["after"], completion=data["completion"])
            },
    )
    predictions.append(prediction)
    # count += 1
    # if count == 2:
        # break

# Code Summarization: 1-stage
구현할 때부터 짧게 구현해달라고 요청

In [ ]:
dataset = load_dataset("openai_humaneval", split="test")
dataset

In [34]:
template = """Implement the code below as short as possible.
[PYTHON]
{}
[/PYTHON]"""
print(template.format(dataset[0]["prompt"]))

Implement the code below as short as possible.
[PYTHON]
from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """

[/PYTHON]


In [37]:
predictions = []
for example in dataset:
    prediction = replicate.predictions.create(
        version=version,
        input={
                "top_k": 50,
                "top_p": 1.0,
                "max_tokens": 500,
                "temperature": 0,
                "system_prompt": "Responses should be written in Python.",
                "prompt": template.format(example["prompt"]),
                "repeat_penalty": 1.1,
                "presence_penalty": 0,
                "frequency_penalty": 0
            },
    )
    predictions.append(prediction)

# Self-Planning

## Planning phase

In [396]:
template = """[INST] Your task is to 
Plan the steps to implement the following Python function. 
Put a number in front of each step.
Response with ONLY natural language plan steps, with numbering.
DO NOT print python function. Print ONLY plans.
[\INST]

[PYTHON]
{completion}
[\PYTHON]
"""

In [397]:
print(template.format(completion=dataset[30]["prompt"].strip()))

[INST] Your task is to 
Plan the steps to implement the following Python function. 
Put a number in front of each step.
Response with ONLY natural language plan steps, with numbering.
DO NOT print python function. Print ONLY plans.
[\INST]

[PYTHON]
def get_positive(l: list):
    """Return only positive numbers in the list.
    >>> get_positive([-1, 2, -4, 5, 6])
    [2, 5, 6]
    >>> get_positive([5, 3, -5, 2, -3, 3, 9, 0, 123, 1, -10])
    [5, 3, 2, 3, 9, 123, 1]
    """
[\PYTHON]



In [399]:
predictions = []
count = 0
for data in dataset:
    prediction = replicate.predictions.create(
        version=version,
        input={
                **params,
                "prompt": template.format(completion=data["prompt"].strip()),
                "system_prompt": "Responses should be written in natural language."
            },
    )
    predictions.append(prediction)

# Manage status

In [400]:
all_succeeded = False

while not all_succeeded:
    all_succeeded = True
    for prediction in predictions:
        if prediction.status == "succeeded":
            continue # Already succeeded, skip

        prediction.reload()
        if prediction.status == "succeeded": # succeeded임에도 계속 running으로 떠서 비용이 부과되는 문제가 있음. 수동으로 cancel
            prediction.cancel()
        else: 
            print(f"{prediction.id}: {prediction.status}")
            all_succeeded = False
        
        print(f"{prediction.id}: {prediction.status}")
    print("-----")

kppq3llbkx65ybfkg5wnapwt5e: succeeded
2khpdrlblt65zgmymbsil66gmi: succeeded
pvwotn3b27vnkzv3qvfpvob4vu: succeeded
ti2kmgdbqvzpstyue4neyluv5e: succeeded
ej3rlhlbv63sxtcazsltyyrani: succeeded
dm6bhvlburgkfxlmfjxcemgwxy: succeeded
v23urmlb3kb6wlggimcjmog7iy: succeeded
jiegg63bnum7chnj5prmmdqg4m: processing
jiegg63bnum7chnj5prmmdqg4m: processing
nsxq5olb5ecfntfsgb2upqqwcy: starting
nsxq5olb5ecfntfsgb2upqqwcy: starting
pfz3yg3bs5yobemy3m27kvunxa: starting
pfz3yg3bs5yobemy3m27kvunxa: starting
3e4dk4tbwzpwzutqeuwp2oztya: starting
3e4dk4tbwzpwzutqeuwp2oztya: starting
fhtpg73bbfw35pyjqxebqphcei: starting
fhtpg73bbfw35pyjqxebqphcei: starting
6lyerilbbrbwh4g42og4psn62q: starting
6lyerilbbrbwh4g42og4psn62q: starting
thw43ddbgzynavde2sh4k7xl5u: starting
thw43ddbgzynavde2sh4k7xl5u: starting
477j5g3bsv7wi6db4b3c6q4thm: starting
477j5g3bsv7wi6db4b3c6q4thm: starting
cueuxudbiw6o525z4oozgxjhli: starting
cueuxudbiw6o525z4oozgxjhli: starting
auxik6lbmxzkm62rb2dultnvxi: starting
auxik6lbmxzkm62rb2dultnvxi:

# Convert to JSON

In [401]:
for prediction in predictions:
    prediction.reload()

In [402]:
predictions

[Prediction(id='kppq3llbkx65ybfkg5wnapwt5e', model='meta/codellama-34b-instruct', version='b17fdb44c843000741367ae3d73e2bb710d7428a662238ddebbf4302db2b5422', status='succeeded', input={'frequency_penalty': 0, 'max_tokens': 300, 'presence_penalty': 0, 'prompt': '[INST] Your task is to \nPlan the steps to implement the following Python function. \nPut a number in front of each step.\nResponse with ONLY natural language plan steps, with numbering.\nDO NOT print python function. Print ONLY plans.\n[\\INST]\n\n[PYTHON]\nfrom typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n[\\PYTHON]\n', 'repeat_penalty': 1.1, 'system_prompt': 'Responses should be written in natural language.', 'temperature'

In [409]:
results = []
for i, prediction in enumerate(predictions):
    results.append({
        "task_id": dataset["task_id"][i],
        "completion": "".join(prediction.output)
    })
results

[{'task_id': 'HumanEval/0',
  'completion': '  \n\nHere is a natural language plan for implementing the Python function:\n\n1. Sort the list of numbers in ascending order.\n2. Iterate through the sorted list and check if any two adjacent elements are closer to each other than the given threshold.\n3. If two adjacent elements meet the condition, return True.\n4. If no two adjacent elements meet the condition, return False.'},
 {'task_id': 'HumanEval/1',
  'completion': '  \n1. Define a function `separate_paren_groups` that takes a string argument `paren_string`.\n2. Initialize an empty list `result` to store the separated groups.\n3. Use a loop to iterate over each character in `paren_string`.\n4. If the current character is an opening parenthesis, start a new group and add it to `result`.\n5. If the current character is a closing parenthesis, end the current group.\n6. If the current character is not a parenthesis, ignore it.\n7. Return `result` at the end of the function.'},
 {'task_i

In [407]:
with jsonlines.open("raw.json", mode="w") as writer:
    for result in results:
        writer.write(result)

In [381]:
for result in results:
    # STrings are wrapped with [PYTHON] [/PYTHON] tags. Remove them.
    try:
        assert "[PYTHON]" in result["completion"]
    except AssertionError:
        print(result["task_id"])
        print(result["completion"])
        result["completion"] = result["completion"].strip()
        continue
    if "[PYTHON]" in result["completion"]:
        result["completion"] = result["completion"].split("[PYTHON]")[1].split("[/PYTHON]")[0].strip()

results

HumanEval/89
  
def encrypt(s):
    alphabet = "abcdefghijklmnopqrstuvwxyz"
    new_alphabet = alphabet[2:] + alphabet[:2]
    table = str.maketrans(alphabet, new_alphabet)
    return s.translate(table)
HumanEval/93
  
def encode(message):
    vowels = ['a', 'e', 'i', 'o', 'u']
    encoded = ''
    for char in message:
        if char.isalpha():
            encoded += chr(ord(char) + 1) if char.islower() else chr(ord(char) - 1)
        else:
            encoded += char
    
    for i, char in enumerate(encoded):
        if char in vowels:
            encoded = encoded[:i] + chr(ord(char) + 2) + encoded[i+1:]
    
    return encoded
HumanEval/118
  
def get_closest_vowel(word):
    vowels = "aeiou"
    for i in range(len(word)-1, 0, -1):
        if word[i] in vowels:
            return word[i]
    return ""
HumanEval/160
  
def do_algebra(operator, operand):
    result = operand[0]
    for op, num in zip(operator, operand[1:]):
        if op == '+':
            result += num
        eli

[{'task_id': 'HumanEval/0',
  'completion': 'def has_close_elements(numbers: List[float], threshold: float) -> bool:\n    return any(abs(a - b) <= threshold for a, b in zip(numbers, numbers[1:]))'},
 {'task_id': 'HumanEval/1',
  'completion': "def separate_paren_groups(paren_string):\n    return re.findall(r'\\(.*?\\)', paren_string)"},
 {'task_id': 'HumanEval/2',
  'completion': 'def truncate_number(number: float) -> float:\n    return number % 1'},
 {'task_id': 'HumanEval/3',
  'completion': 'def below_zero(operations: List[int]) -> bool:\n    return any(operation < 0 for operation in operations)'},
 {'task_id': 'HumanEval/4',
  'completion': 'def minLengthAfterRemovals(self, nums: List[int]) -> int:\n    scount = Counter(nums)\n    arr = sorted([scount[num] for num in sorted(scount.keys())], key=lambda x: -x)    \n    \n    return (arr[0] - sum(arr[1:])) if arr[0] > sum(arr[1:]) else sum(arr) % 2'},
 {'task_id': 'HumanEval/5',
  'completion': 'def intersperse(numbers: List[int], del

In [ ]:
import re
from pprint import pprint

for result in results:
    # STrings are wrapped with [PYTHON] [/PYTHON] tags. Remove them.
    try:
        assert "1." in result["completion"]
    except AssertionError as e:
        print(result["task_id"])
        print(result["completion"])
        result["completion"] = "" # Set to empty string if no plan is found.
        continue
    
    lines = result["completion"].split("\n")
    # Only keep the lines that matches regex "\d."
    lines = [line for line in lines if re.match(".*\.", line[:5])]
    result["completion"] = "\n".join(lines)
    result["completion"] = result["completion"].strip()
results

In [411]:
for result in results:
    print(result["completion"])
    print("-" * 80)

1. Sort the list of numbers in ascending order.
2. Iterate through the sorted list and check if any two adjacent elements are closer to each other than the given threshold.
3. If two adjacent elements meet the condition, return True.
4. If no two adjacent elements meet the condition, return False.
--------------------------------------------------------------------------------
1. Define a function `separate_paren_groups` that takes a string argument `paren_string`.
2. Initialize an empty list `result` to store the separated groups.
3. Use a loop to iterate over each character in `paren_string`.
4. If the current character is an opening parenthesis, start a new group and add it to `result`.
5. If the current character is a closing parenthesis, end the current group.
6. If the current character is not a parenthesis, ignore it.
7. Return `result` at the end of the function.
--------------------------------------------------------------------------------
1. Start by defining the function w

In [412]:
experiment_name = "planning"
path_json = f"predictions/prediction_{experiment_name}.jsonl"
path_txt = f"predictions/prediction_{experiment_name}.txt"

In [413]:
with jsonlines.open(path_json, mode="w") as writer:
    for result in results:
        writer.write(result)

In [414]:
with open(path_txt, "w") as writer:
    for result in results:
        writer.write(f"{result['task_id']}\n\n")
        writer.write(f"{result['completion']}\n\n")
        writer.write('-'*50 + '\n')

In [261]:
dir = "pass@k_eval/predictions"
experiment = "summary_2"
total_code_length = 0
with jsonlines.open(os.path.join(dir, f"prediction_{experiment}.jsonl")) as reader:
    for obj in reader:
        total_code_length += len(obj["completion"])

print("Total: ", total_code_length)
print("Average: ", total_code_length / len(dataset))

FileNotFoundError: [Errno 2] No such file or directory: 'pass@k_eval/predictions/prediction_summary_2.jsonl'

# Emergency Cancel

In [ ]:
# 실수했을 때 모든 prediction을 취소 (돈 아까워서)
for prediction in predictions:
    prediction.cancel()

In [417]:
preds = []
with open("predictions/prediction_summary_2_1shot_2.txt") as reader:
    full = "".join(reader.readlines())
full

'HumanEval/0\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    return any(abs(a - b) <= threshold for a, b in zip(numbers, numbers[1:]))\n\n--------------------------------------------------\nHumanEval/1\n\ndef separate_paren_groups(paren_string):\n    return re.findall(r\'\\(.*?\\)\', paren_string)\n\n--------------------------------------------------\nHumanEval/2\n\ndef truncate_number(number: float) -> float:\n    return number % 1\n\n--------------------------------------------------\nHumanEval/3\n\ndef below_zero(operations: List[int]) -> bool:\n    return any(operation < 0 for operation in operations)\n\n--------------------------------------------------\nHumanEval/4\n\ndef minLengthAfterRemovals(self, nums: List[int]) -> int:\n    scount = Counter(nums)\n    arr = sorted([scount[num] for num in sorted(scount.keys())], key=lambda x: -x)    \n    \n    return (arr[0] - sum(arr[1:])) if arr[0] > sum(arr[1:]) else sum(arr) % 2\n\n-----------------------

In [424]:
preds = full.split("-"*50)[:-1]
preds

['HumanEval/0\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    return any(abs(a - b) <= threshold for a, b in zip(numbers, numbers[1:]))\n\n',
 "\nHumanEval/1\n\ndef separate_paren_groups(paren_string):\n    return re.findall(r'\\(.*?\\)', paren_string)\n\n",
 '\nHumanEval/2\n\ndef truncate_number(number: float) -> float:\n    return number % 1\n\n',
 '\nHumanEval/3\n\ndef below_zero(operations: List[int]) -> bool:\n    return any(operation < 0 for operation in operations)\n\n',
 '\nHumanEval/4\n\ndef minLengthAfterRemovals(self, nums: List[int]) -> int:\n    scount = Counter(nums)\n    arr = sorted([scount[num] for num in sorted(scount.keys())], key=lambda x: -x)    \n    \n    return (arr[0] - sum(arr[1:])) if arr[0] > sum(arr[1:]) else sum(arr) % 2\n\n',
 '\nHumanEval/5\n\ndef intersperse(numbers: List[int], delimeter: int) -> List[int]:\n    return [x for pair in zip(numbers[::2], numbers[1::2]) for x in pair] + numbers[-1:]\n\n',
 "\nHumanEval/6\n\nd

In [426]:
type(preds)

list

In [429]:
for i, pred in enumerate(preds):
    preds[i] = re.sub("HumanEval.*\n", "", pred)
    preds[i] = preds[i].strip()
preds

['def has_close_elements(numbers: List[float], threshold: float) -> bool:\n    return any(abs(a - b) <= threshold for a, b in zip(numbers, numbers[1:]))',
 "def separate_paren_groups(paren_string):\n    return re.findall(r'\\(.*?\\)', paren_string)",
 'def truncate_number(number: float) -> float:\n    return number % 1',
 'def below_zero(operations: List[int]) -> bool:\n    return any(operation < 0 for operation in operations)',
 'def minLengthAfterRemovals(self, nums: List[int]) -> int:\n    scount = Counter(nums)\n    arr = sorted([scount[num] for num in sorted(scount.keys())], key=lambda x: -x)    \n    \n    return (arr[0] - sum(arr[1:])) if arr[0] > sum(arr[1:]) else sum(arr) % 2',
 'def intersperse(numbers: List[int], delimeter: int) -> List[int]:\n    return [x for pair in zip(numbers[::2], numbers[1::2]) for x in pair] + numbers[-1:]',
 "def parse_nested_parens(paren_string: str) -> List[int]:\n    def get_max_level(s: str) -> int:\n        level = 0\n        max_level = 0\n   

In [430]:
with jsonlines.open("predictions/prediction_summary_2_1shot_2.jsonl", mode="w") as writer:
    for i, pred in enumerate(preds):
        writer.write({"task_id": dataset[i]["task_id"], "completion": pred})